# Preparation

In [55]:
pip install gensim --user

Note: you may need to restart the kernel to use updated packages.


In [56]:
#Import

import nltk
import gensim
import pandas as pd

from sklearn.metrics import precision_recall_fscore_support

In [57]:
# Gensim import dan load pre-trained idwiki language model

path = '../id_wiki/idwiki_word2vec_300.model'
id_w2v = gensim.models.word2vec.Word2Vec.load(path)
print(id_w2v.wv.most_similar('tes'))

[('pengujian', 0.7212876677513123), ('uji', 0.7056494951248169), ('pemeriksaan', 0.6564321517944336), ('ujian', 0.5918323993682861), ('percobaan', 0.5823646187782288), ('eksperimen', 0.5750775933265686), ('test', 0.550246000289917), ('serologis', 0.5315841436386108), ('verifikasi', 0.5280280113220215), ('ujicoba', 0.5256417393684387)]


# Prepare Dataset

In [58]:
#Prepare Dataset
yelp = pd.read_csv('dataset.csv', sep=',', header = 0)
del yelp['No']
yelp = yelp.dropna()
yelp.head()

,jenis,uraian,pertanyaan,opsi1,opsi2,opsi3,opsi4,kunci_jawaban,kategori
0,Makna kata/istilah,Padi yang luas menguning adalah hasil tanam ya...,Makna kata jasa pada paragraf di atas adalah,Pelayanan yang terbaik bagi kita.\t,Manfaat yang melimpah bagi kita\t,Perbuatan yang berguna bagi orang lain\t,Jerih payah yang sangat menguntungkan,Perbuatan yang berguna bagi orang lain\t,Mudah
1,Antonim/sinonim,Suara radio tetangga sebelah sangat keras pada...,Antonim kata keras pada kalimat tersebut adalah,pelan\t,lunak\t,kaku\t,kencang,pelan\t,Mudah
2,Informasi tersurat teks,Pembiasaan Hidup Sehat Sejak KecilPola hidup s...,Bagaimana cara memenuhi gizi anak pada usia ba...,Menambah gizi anak seiring bertambahnya usia a...,"Makan makanan bergizi, istirahat cukup, dan ol...",Mempunyai kekebalan yang baik terhadap seranga...,Pemenuhan gizi dengan pemberian ASI saat anak...,Pemenuhan gizi dengan pemberian ASI saat anak...,Sedang
3,Informasi tersirat teks,Mulai tahun 2016 kuota haji di Indonesssia dit...,Berdasarkan data kuota jumlah jamaah haji untu...,77,77.7,78,"77,800",77,Sedang
4,Informasi tersurat teks,Kebiasaan makan makanan bergizi dengan kadar s...,Kalimat dalam paragraf di atas yang tidak padu...,Kebutuhan gizi anak akan semakin bertambah sei...,Kebutuhan gizi anak baru terpenuhi saat anak b...,Tubuh anak mudah terserang berbagai macam peny...,Pertumbuhan dan perkembangan anak akan semakin...,Tubuh anak mudah terserang berbagai macam peny...,Sedang


In [59]:
#Data Cleaning
yelp = yelp.replace('\n',' ', regex=True)
yelp = yelp.replace('\t','', regex=True)
yelp = yelp.replace('"',' " ', regex=True)
yelp.head()

,jenis,uraian,pertanyaan,opsi1,opsi2,opsi3,opsi4,kunci_jawaban,kategori
0,Makna kata/istilah,Padi yang luas menguning adalah hasil tanam ya...,Makna kata jasa pada paragraf di atas adalah,Pelayanan yang terbaik bagi kita.,Manfaat yang melimpah bagi kita,Perbuatan yang berguna bagi orang lain,Jerih payah yang sangat menguntungkan,Perbuatan yang berguna bagi orang lain,Mudah
1,Antonim/sinonim,Suara radio tetangga sebelah sangat keras pada...,Antonim kata keras pada kalimat tersebut adalah,pelan,lunak,kaku,kencang,pelan,Mudah
2,Informasi tersurat teks,Pembiasaan Hidup Sehat Sejak KecilPola hidup s...,Bagaimana cara memenuhi gizi anak pada usia ba...,Menambah gizi anak seiring bertambahnya usia a...,"Makan makanan bergizi, istirahat cukup, dan ol...",Mempunyai kekebalan yang baik terhadap seranga...,Pemenuhan gizi dengan pemberian ASI saat anak...,Pemenuhan gizi dengan pemberian ASI saat anak...,Sedang
3,Informasi tersirat teks,Mulai tahun 2016 kuota haji di Indonesssia dit...,Berdasarkan data kuota jumlah jamaah haji untu...,77,77.7,78,"77,800",77,Sedang
4,Informasi tersurat teks,Kebiasaan makan makanan bergizi dengan kadar s...,Kalimat dalam paragraf di atas yang tidak padu...,Kebutuhan gizi anak akan semakin bertambah sei...,Kebutuhan gizi anak baru terpenuhi saat anak b...,Tubuh anak mudah terserang berbagai macam peny...,Pertumbuhan dan perkembangan anak akan semakin...,Tubuh anak mudah terserang berbagai macam peny...,Sedang


In [60]:
df = pd.DataFrame()

#Kolom teks merupakan gabungan dari kolom jenis soal, uraian soal, pertanyaan soal, opsi1 soal, opsi2 soal, opsi3 soal, opsi4 soal, dan kunci jawaban soal
df['teks'] = yelp['jenis'].map(str) + ' ' + yelp['uraian'].map(str)+ ' ' + yelp['pertanyaan'].map(str)+ ' ' + yelp['opsi1'].map(str)+ ' ' + yelp['opsi2'].map(str)+ ' ' + yelp['opsi3'].map(str)+ ' ' + yelp['opsi4'].map(str)+ ' ' + yelp['kunci_jawaban'].map(str)

#kolom kategori merupakan kolom kategori kesulitan soal
df['kategori'] = yelp['kategori']

yelp = yelp.dropna()
print(df)

                                                  teks kategori
0    Makna kata/istilah Padi yang luas menguning ad...    Mudah
1    Antonim/sinonim Suara radio tetangga sebelah s...    Mudah
2    Informasi tersurat teks Pembiasaan Hidup Sehat...   Sedang
3    Informasi tersirat teks Mulai tahun 2016 kuota...   Sedang
4    Informasi tersurat teks Kebiasaan makan makana...   Sedang
..                                                 ...      ...
413  Tanda baca Merdeka, merdeka!  " Teruskan geril...    Mudah
414  Kata bentukan Tebing Breksi adalah salah satu ...    Sulit
415  Tanda baca  " Sebaiknya kita membawa buah tang...    Mudah
416  Istilah/kata Permasalahan yang dihadapi Nia se...    Sulit
417  Tanda baca Joya berkata,  " Wow, Alangkah inda...    Mudah

[418 rows x 2 columns]


# Data Pre-processing

In [61]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\fikri\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

## Stopwords & Embedding Vectorization(Word2Vec)

In [62]:
#Pembentukan dataframe untuk vektorisasi embedding

docs_vectors = pd.DataFrame() # creating empty final dataframe
stopwords = nltk.corpus.stopwords.words('indonesian') # removing stop words
for doc in yelp['pertanyaan'].str.lower().str.replace('[^a-z ]', ''): # looping through each document and cleaning it
    temp = pd.DataFrame()  # creating a temporary dataframe(store value for 1st doc & for 2nd doc remove the details of 1st & proced through 2nd and so on..)
    for word in doc.split(' '): # looping through each word of a single document and spliting through space
        if word not in stopwords: # if word is not present in stopwords then (try)
            try:
                word_vec = id_w2v.wv[word] # if word is present in embeddings(id_wikidata(300)) then proceed
                temp = temp.append(pd.Series(word_vec), ignore_index = True) # if word is present then append it to temporary dataframe
            except:
                pass
    doc_vector = temp.mean() # take the average of each column(w0, w1, w2,........w300)
    docs_vectors = docs_vectors.append(doc_vector, ignore_index = True) # append each document value to the final dataframe
docs_vectors.shape

C:\Users\fikri\AppData\Local\Temp/ipykernel_5920/1651433560.py:5: FutureWarning: The default value of regex will change from True to False in a future version.
  for doc in yelp['pertanyaan'].str.lower().str.replace('[^a-z ]', ''): # looping through each document and cleaning it


(418, 300)

In [63]:
#Pengecekan terhadap null values pada dataframe docs_vector

pd.isnull(docs_vectors).sum().sum()

0

In [64]:
#transformasi kolom kategori yang semula teks menuju bentuk angka

df['kategori'] = df['kategori'].astype("category")
docs_vectors['label'] = df['kategori'].cat.codes
docs_vectors = docs_vectors.dropna()

# Generate CSV Dataset for Evaluation

In [65]:
docs_vectors.to_csv('dataset_eval_w2v.csv', index=False, encoding='utf-8')

print("Dataset for evauation successfully generated !")

Dataset for evauation successfully generated !


# Train & Test Dataset Split

In [66]:
#Persiapan evaluasi - split dataset dan pemilihan teknik classifier

from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier

df_eval = dataset = pd.read_csv('dataset_eval_w2v.csv', header=0, engine='python')

train_x, test_x, train_y, test_y = train_test_split(df_eval.drop('label', axis = 1),
                                                   df_eval['label'],
                                                   test_size = 0.15, random_state=42)
train_x.shape, train_y.shape, test_x.shape, test_y.shape

((355, 300), (355,), (63, 300), (63,))

# Single Fold Evaluation

In [67]:
#Evaluasi akurasi klasifikasi

model = RandomForestClassifier(n_estimators=300)
model.fit(train_x, train_y)
test_pred = model.predict(test_x)
from sklearn.metrics import accuracy_score
accuracy_score(test_y, test_pred)

0.6666666666666666

In [68]:
#predicted y
y_pred = model.predict(test_x)

In [69]:
from sklearn import metrics

print(metrics.classification_report(test_y, y_pred, target_names=['Mudah', 'Sedang', 'Sulit']))

              precision    recall  f1-score   support

       Mudah       0.73      0.92      0.81        24
      Sedang       0.45      0.38      0.42        13
       Sulit       0.68      0.58      0.62        26

    accuracy                           0.67        63
   macro avg       0.62      0.63      0.62        63
weighted avg       0.65      0.67      0.65        63



# Machine Learning Classification Evaluation

## Random Forest Classifier 

In [70]:
from sklearn.ensemble import RandomForestClassifier

acc_temp = 0
prec_temp = 0
rec_temp = 0
f1_temp = 0
n_fold = 10

for i in range(n_fold):
    model = RandomForestClassifier(n_estimators = 300)
    model.fit(train_x, train_y)

    test_pred = model.predict(test_x)
    from sklearn.metrics import accuracy_score
    temp_res = accuracy_score(test_y, test_pred)
    acc_temp+= temp_res
    
    precision, recall, f_value, support = precision_recall_fscore_support(test_y, test_pred, average='macro')
    
    prec_temp+=precision
    rec_temp += recall
    f1_temp += f_value
    
    print(f'Fold {i+1} Eval...   | ACC : {temp_res} | PREC : {precision} | REC : {recall} | F1-SCORE : {f_value} | SUPP : {support}')

res = acc_temp/n_fold
prec_res = prec_temp/n_fold
rec_res = rec_temp/n_fold
f1_res = f1_temp/n_fold

print(f'Average Accuracy Score({n_fold} Times Evaluation) : {res}')
print(f'Average Precision Score({n_fold} Times Evaluation) : {prec_res}')
print(f'Average Recall Score({n_fold} Times Evaluation) : {rec_res}')
print(f'Average F1 Score({n_fold} Times Evaluation) : {f1_res}')

Fold 1 Eval...   | ACC : 0.6984126984126984 | PREC : 0.6706304967174533 | REC : 0.6762820512820512 | F1-SCORE : 0.67168405823868 | SUPP : None
Fold 2 Eval...   | ACC : 0.6984126984126984 | PREC : 0.6476068376068377 | REC : 0.6517094017094017 | F1-SCORE : 0.6488888888888887 | SUPP : None
Fold 3 Eval...   | ACC : 0.6984126984126984 | PREC : 0.6581196581196581 | REC : 0.6645299145299145 | F1-SCORE : 0.6605128205128206 | SUPP : None
Fold 4 Eval...   | ACC : 0.6984126984126984 | PREC : 0.6476068376068377 | REC : 0.6517094017094017 | F1-SCORE : 0.6488888888888887 | SUPP : None
Fold 5 Eval...   | ACC : 0.6984126984126984 | PREC : 0.6625641025641026 | REC : 0.6634615384615384 | F1-SCORE : 0.6622222222222223 | SUPP : None
Fold 6 Eval...   | ACC : 0.7301587301587301 | PREC : 0.6753246753246754 | REC : 0.6784188034188036 | F1-SCORE : 0.6737606837606838 | SUPP : None
Fold 7 Eval...   | ACC : 0.6825396825396826 | PREC : 0.6368376068376068 | REC : 0.6378205128205128 | F1-SCORE : 0.636996336996337 | 

## Naive Bayes Classifier 

In [71]:
from sklearn.naive_bayes import GaussianNB

acc_temp = 0
prec_temp = 0
rec_temp = 0
f1_temp = 0
n_fold = 10

for i in range(n_fold):
    model = GaussianNB()
    model.fit(train_x, train_y)

    test_pred = model.predict(test_x)
    from sklearn.metrics import accuracy_score
    temp_res = accuracy_score(test_y, test_pred)
    acc_temp+= temp_res
    
    precision, recall, f_value, support = precision_recall_fscore_support(test_y, test_pred, average='macro')
    
    prec_temp+=precision
    rec_temp += recall
    f1_temp += f_value
    
    print(f'Fold {i+1} Eval...   | ACC : {temp_res} | PREC : {precision} | REC : {recall} | F1-SCORE : {f_value} | SUPP : {support}')

res = acc_temp/n_fold
prec_res = prec_temp/n_fold
rec_res = rec_temp/n_fold
f1_res = f1_temp/n_fold

print(f'Average Accuracy Score({n_fold} Times Evaluation) : {res}')
print(f'Average Precision Score({n_fold} Times Evaluation) : {prec_res}')
print(f'Average Recall Score({n_fold} Times Evaluation) : {rec_res}')
print(f'Average F1 Score({n_fold} Times Evaluation) : {f1_res}')

Fold 1 Eval...   | ACC : 0.6031746031746031 | PREC : 0.5816239316239317 | REC : 0.5844017094017094 | F1-SCORE : 0.5768198521821711 | SUPP : None
Fold 2 Eval...   | ACC : 0.6031746031746031 | PREC : 0.5816239316239317 | REC : 0.5844017094017094 | F1-SCORE : 0.5768198521821711 | SUPP : None
Fold 3 Eval...   | ACC : 0.6031746031746031 | PREC : 0.5816239316239317 | REC : 0.5844017094017094 | F1-SCORE : 0.5768198521821711 | SUPP : None
Fold 4 Eval...   | ACC : 0.6031746031746031 | PREC : 0.5816239316239317 | REC : 0.5844017094017094 | F1-SCORE : 0.5768198521821711 | SUPP : None
Fold 5 Eval...   | ACC : 0.6031746031746031 | PREC : 0.5816239316239317 | REC : 0.5844017094017094 | F1-SCORE : 0.5768198521821711 | SUPP : None
Fold 6 Eval...   | ACC : 0.6031746031746031 | PREC : 0.5816239316239317 | REC : 0.5844017094017094 | F1-SCORE : 0.5768198521821711 | SUPP : None
Fold 7 Eval...   | ACC : 0.6031746031746031 | PREC : 0.5816239316239317 | REC : 0.5844017094017094 | F1-SCORE : 0.5768198521821711

## SVM Classifier 

In [72]:
from sklearn.svm import SVC

acc_temp = 0
prec_temp = 0
rec_temp = 0
f1_temp = 0
n_fold = 10

for i in range(n_fold):
    model = SVC(kernel='linear')
    model.fit(train_x, train_y)

    test_pred = model.predict(test_x)
    from sklearn.metrics import accuracy_score
    temp_res = accuracy_score(test_y, test_pred)
    acc_temp+= temp_res
    
    precision, recall, f_value, support = precision_recall_fscore_support(test_y, test_pred, average='macro')
    
    prec_temp+=precision
    rec_temp += recall
    f1_temp += f_value
    
    print(f'Fold {i+1} Eval...   | ACC : {temp_res} | PREC : {precision} | REC : {recall} | F1-SCORE : {f_value} | SUPP : {support}')

res = acc_temp/n_fold
prec_res = prec_temp/n_fold
rec_res = rec_temp/n_fold
f1_res = f1_temp/n_fold

print(f'Average Accuracy Score({n_fold} Times Evaluation) : {res}')
print(f'Average Precision Score({n_fold} Times Evaluation) : {prec_res}')
print(f'Average Recall Score({n_fold} Times Evaluation) : {rec_res}')
print(f'Average F1 Score({n_fold} Times Evaluation) : {f1_res}')

Fold 1 Eval...   | ACC : 0.5873015873015873 | PREC : 0.5520202020202021 | REC : 0.547008547008547 | F1-SCORE : 0.5382659547419274 | SUPP : None
Fold 2 Eval...   | ACC : 0.5873015873015873 | PREC : 0.5520202020202021 | REC : 0.547008547008547 | F1-SCORE : 0.5382659547419274 | SUPP : None
Fold 3 Eval...   | ACC : 0.5873015873015873 | PREC : 0.5520202020202021 | REC : 0.547008547008547 | F1-SCORE : 0.5382659547419274 | SUPP : None
Fold 4 Eval...   | ACC : 0.5873015873015873 | PREC : 0.5520202020202021 | REC : 0.547008547008547 | F1-SCORE : 0.5382659547419274 | SUPP : None
Fold 5 Eval...   | ACC : 0.5873015873015873 | PREC : 0.5520202020202021 | REC : 0.547008547008547 | F1-SCORE : 0.5382659547419274 | SUPP : None
Fold 6 Eval...   | ACC : 0.5873015873015873 | PREC : 0.5520202020202021 | REC : 0.547008547008547 | F1-SCORE : 0.5382659547419274 | SUPP : None
Fold 7 Eval...   | ACC : 0.5873015873015873 | PREC : 0.5520202020202021 | REC : 0.547008547008547 | F1-SCORE : 0.5382659547419274 | SUPP